# Install Detectron2 Dependencies

In [9]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-iw5ugszr
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-iw5ugszr
1.5.0+cu101 False
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [10]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html


In [11]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

# Import and Register Custom Detectron2 Data

In [12]:
# #visualize training data
# my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
# dataset_dicts = DatasetCatalog.get("my_dataset_train")

# import random
# from detectron2.utils.visualizer import Visualizer

# for d in random.sample(dataset_dicts, 3):
#     img = cv2.imread(d["file_name"])
#     visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
#     vis = visualizer.draw_dataset_dict(d)
#     cv2_imshow(vis.get_image()[:, :, ::-1])

# load Custom Detectron2 Detector

# Inference with Detectron2 Saved Weights



In [1]:
%ls ./output/

Parameter format not correct - "utput".


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp '/content/gdrive/MyDrive/colab src/output' '/content/'

cp: -r not specified; omitting directory '/content/gdrive/MyDrive/colab src/output'


In [18]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml'))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # Set threshold for this model
cfg.MODEL.WEIGHTS = '/content/gdrive/MyDrive/colab src/output/model_final.pth' # Set path model .pth
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.MODEL.DEVICE='cpu'
predictor = DefaultPredictor(cfg)

In [23]:
from detectron2.utils.visualizer import ColorMode
import glob

res = []
count=0

for imageName in glob.glob('/content/sample_data/*tif'):
  count += 1
  print(count, imageName)
  im = cv2.imread(imageName)
  outputs = predictor(im)
  res.append({'file_name': imageName, **outputs})

print(res)

1 /content/sample_data/6a381bb657efa536ec4b11dd3a608820.tif
2 /content/sample_data/eb09d89ba31fed396d5a5a8d8a9d1828.tif
3 /content/sample_data/69a1fa63b5ef242c6f358c2975e3ac35.tif
4 /content/sample_data/27124678eef95d8801ac23d9f5f1ad55.tif
5 /content/sample_data/bfe582366beebd7f8848bbf53d6304be_3.tif
6 /content/sample_data/3832e7dac6c2fc6750c7ad7aadda9a95.tif
7 /content/sample_data/aa18781ad618f6aae949142d51d13b21_2.tif
8 /content/sample_data/75afebf5f040177e9b1e35d8119a04c3_2.tif
9 /content/sample_data/c1c43fe5d79a70bf949c05e5b8ed267d_1.tif
10 /content/sample_data/1038a29f20f610df2e01b36f51bc9a31.tif
11 /content/sample_data/e17e418919f88f7155b85f2bf58e7975.tif
12 /content/sample_data/4b520da4e68716484a7374a42bf7e2a9_2.tif
13 /content/sample_data/ebfea1a0350207555bb9ce733d3ea7fd_3.tif
14 /content/sample_data/bcef979fdc530eba01f536f47878813f_1.tif
15 /content/sample_data/ca2a234d58852de50b07a6a216370b6d_7.tif
16 /content/sample_data/bcef979fdc530eba01f536f47878813f_2.tif
17 /content/sam

In [25]:
with open('results.txt', 'w') as f:
  for item in res:
        f.write("%s\n" % item)

In [75]:
parsed_res = []
for item in res:
  parsed_res.append((item['file_name'][21:-4], 1 if len(item['instances'].get('pred_boxes')) > 0 else 0))

print(parsed_res)

[('6a381bb657efa536ec4b11dd3a608820', 1), ('eb09d89ba31fed396d5a5a8d8a9d1828', 1), ('69a1fa63b5ef242c6f358c2975e3ac35', 1), ('27124678eef95d8801ac23d9f5f1ad55', 1), ('bfe582366beebd7f8848bbf53d6304be_3', 0), ('3832e7dac6c2fc6750c7ad7aadda9a95', 1), ('aa18781ad618f6aae949142d51d13b21_2', 0), ('75afebf5f040177e9b1e35d8119a04c3_2', 1), ('c1c43fe5d79a70bf949c05e5b8ed267d_1', 0), ('1038a29f20f610df2e01b36f51bc9a31', 1), ('e17e418919f88f7155b85f2bf58e7975', 1), ('4b520da4e68716484a7374a42bf7e2a9_2', 1), ('ebfea1a0350207555bb9ce733d3ea7fd_3', 1), ('bcef979fdc530eba01f536f47878813f_1', 1), ('ca2a234d58852de50b07a6a216370b6d_7', 1), ('bcef979fdc530eba01f536f47878813f_2', 1), ('6b816984ec5fbd0541836da977d4570d_3', 0), ('4cb366eee206bc9f0b48c0c8d7cc94ae', 1), ('dc703955d59f63982f81892900824cc4_2', 0), ('d0813ddce0c72c0fb0c33257903d518d_5', 0), ('ad729ba5891d6199f01d12d59086bb86_1', 0), ('741b84cb122cba80c73c8ef585f7d942_1', 0), ('b76c113a590bac5583eb93912252fd4f_2', 1), ('14cff1e5fc1ede8bd192912a

In [62]:
with open('parsed_results.txt', 'w') as f:
  parsed_res

In [63]:
import pandas

In [77]:
df = pandas.DataFrame(columns=['Id','Expected'], data=parsed_res)
df

,Id,Expected
0,6a381bb657efa536ec4b11dd3a608820,1
1,eb09d89ba31fed396d5a5a8d8a9d1828,1
2,69a1fa63b5ef242c6f358c2975e3ac35,1
3,27124678eef95d8801ac23d9f5f1ad55,1
4,bfe582366beebd7f8848bbf53d6304be_3,0
...,...,...
246,081a5d93952b57ecde40bc02b8b70e67_2,1
247,0148e4778e6d8b22a8980bf4fc89351b_25,0
248,6103ce8203bae073bc7fbe4d839442e6,1
249,2901d78cd3470b6283bfdc8ccb41538a_10,1


In [80]:
compression_opts = dict(method='zip',
                        archive_name='out.csv') 
df.to_csv('out.csv', index=False)

In [129]:
res_with_bbox = []
for item in res:
  #res_with_bbox.append((item['file_name'][21:-4], 1 if len(item['instances'].get('pred_boxes')) > 0 else 0))
  res_with_bbox.append((item['file_name'][21:-4], item['instances'].pred_boxes.tensor.cpu().numpy().tolist(), item['instances'].scores.cpu().numpy().tolist()))

print(res_with_bbox)

[('6a381bb657efa536ec4b11dd3a608820', [[705.2631225585938, 1113.5126953125, 909.6485595703125, 1162.655029296875]], [0.9796600937843323]), ('eb09d89ba31fed396d5a5a8d8a9d1828', [[1465.2926025390625, 2686.25732421875, 1955.6026611328125, 2875.295654296875], [1465.17138671875, 2693.278564453125, 1950.8807373046875, 2786.765380859375]], [0.9861838221549988, 0.7177159190177917]), ('69a1fa63b5ef242c6f358c2975e3ac35', [[617.0125732421875, 1001.3355712890625, 831.1436767578125, 1077.880859375]], [0.9946282505989075]), ('27124678eef95d8801ac23d9f5f1ad55', [[1257.5482177734375, 2313.860107421875, 1654.6566162109375, 2639.489013671875]], [0.9963393211364746]), ('bfe582366beebd7f8848bbf53d6304be_3', [], []), ('3832e7dac6c2fc6750c7ad7aadda9a95', [[1347.7410888671875, 2908.673583984375, 2149.701904296875, 3000.947265625], [1376.476806640625, 2587.397705078125, 1933.6201171875, 2699.5], [1331.98388671875, 2717.79052734375, 1807.275146484375, 2847.15966796875]], [0.9944049119949341, 0.9891216158866882

In [ ]:
for item in res:
  print(item['instances'].scores.cpu().numpy().tolist())


In [ ]:
res_with_bbox_submit = []
for item in res_with_bbox:
  #print(item[1])
  if len(item[1]) == 0:
    res_with_bbox_submit.append((item[0], 0,0,0,0, 0))
  for i,bbox in enumerate(item[1]):
    res_with_bbox_submit.append((item[0], bbox[0], bbox[1], bbox[2], bbox[3], item[2][i]))

res_with_bbox_submit

In [135]:
pandas.DataFrame(columns=['id','xmin','ymin','xmax','ymax','confidence'], data=res_with_bbox_submit).to_csv('with_bbox.csv',index=False)

In [ ]:
# !zip -r /content/output.zip /content/output
# from google.colab import files
# files.download("/content/output.zip")

updating: content/output/ (stored 0%)
updating: content/output/model_final.pth (deflated 6%)
updating: content/output/events.out.tfevents.1634835188.47d788f6b0a9.78.4 (deflated 72%)
updating: content/output/events.out.tfevents.1634828993.47d788f6b0a9.78.0 (deflated 69%)
updating: content/output/my_dataset_test_coco_format.json.lock (stored 0%)
updating: content/output/last_checkpoint (stored 0%)
updating: content/output/events.out.tfevents.1634830632.47d788f6b0a9.78.1 (deflated 5%)
updating: content/output/events.out.tfevents.1634830732.47d788f6b0a9.78.2 (deflated 43%)
updating: content/output/instances_predictions.pth (deflated 57%)
updating: content/output/coco_instances_results.json (deflated 69%)
updating: content/output/metrics.json (deflated 79%)
updating: content/output/events.out.tfevents.1634831801.47d788f6b0a9.78.3 (deflated 70%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>